In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import json
#%matplotlib inline
#pd.set_option('display.max_colwidth', 100)

import hdbscan
import nltk
import numpy as np
import pandas as pd
import random
import re
import spacy
import textacy

from gmplot import gmplot
from mapsplotlib import mapsplot as mplt
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag_sents
from pymprog import *
from sklearn.cluster import DBSCAN
from sklearn.neighbors import DistanceMetric
import pandas as pd

import datetime
from time import strptime

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from sklearn.cluster import KMeans

In [ ]:
fname = "relevant_json.json"
list_tweets = []
count = 0 
sum = 1000
with open(fname,encoding="utf8") as f:
    for line in f:
        try:
            tweet = json.loads(line)
        except:
            pass
        list_tweets.append(tweet)
#         count+=1
#         if count==sum:
#             print(count)
#             sum*=2
# list_tweets[0].keys()

df_relevant = pd.DataFrame(data=[tweet["full_text"] for tweet in list_tweets], columns=['tweets'])
df_relevant['created_at'] = np.array([tweet["created_at"] for tweet in list_tweets])
df_relevant['id'] = np.array([int(tweet["id_str"]) for tweet in list_tweets])
df_relevant['rt'] = np.array([tweet["retweeted"] for tweet in list_tweets])
df_relevant['lang'] = np.array([tweet["lang"] for tweet in list_tweets])
df_relevant['Predicted_Event'] = -1

# df_relevant.shape[0]

fname = "relevant_tweets.tsv"
event_id = pd.read_csv(fname, sep='\t',header=None,names=['Event','id'])

# event_id.shape[0]

df_relevant = df_relevant.merge(event_id, left_on='id', right_on='id')
df_relevant.head(1)

In [ ]:
colnames=['tweets', 'created_at', 'id','rt','lang'] 
df_irrelevant = pd.read_csv('output_1.csv', names=colnames, header=None)
df_irrelevant['Predicted_Event'] = -1
df_irrelevant['Event'] = -1
print(df_irrelevant.shape[0])
df_irrelevant.head(1)

In [ ]:
frames = [df_relevant,df_irrelevant]
df = pd.concat(frames)
# df

import matplotlib.pyplot as plt
# plots number of tweets in each language
counts = df['lang'].value_counts()
plt.bar(counts.index, counts, align='center', alpha=0.5)
plt.xlabel('Tweet Language')
plt.ylabel('# of Tweets')
plt.title('Tweets 2012')

In [ ]:
print("Old df shape:",df.shape[0])
df = df.loc[df["lang"]=='en']
print("New df shape:",df.shape[0])

In [ ]:
df = df.reset_index(drop=True)

def create_date(text):
    temp = text.split(" ")
    x = str(datetime.datetime(int(temp[5]),int(strptime(temp[1],'%b').tm_mon),int(temp[2]))).split()[0]
    return x

df['date'] = df['created_at'].apply(lambda x: create_date(x))
df['time'] =  np.array([tweet.split()[3] for tweet in df["created_at"]])
df['Datetime']= pd.to_datetime(df['date'].apply(str)+' '+df['time'].apply(lambda x: x.split(':')[0]) + df['time'].apply(lambda x: x.split(':')[1]))
df['DateHour'] = pd.to_datetime(df['date'].apply(str)+' '+df['time'].apply(lambda x: x.split(':')[0])+':00')

tableFlag=[]
data_sorted = df.sort_values(by=['DateHour'],inplace=False)
data_sorted = data_sorted.reset_index(drop=True)

list_date = []
for temp in data_sorted["DateHour"]:
    list_date.append(temp)

list_date = list(dict.fromkeys(list_date))
print(len(list_date))
# list_date
timeseries_y = []
for i in range(0,len(list_date)):
    timeseries_y.append(df.loc[df["DateHour"]==list_date[i]].shape[0])
plt.plot(list_date,timeseries_y)

In [ ]:
import spacy
global count10
# result = []
# article = df["tweets"][0]
# spacy_nlp = spacy.load('en')
# document = spacy_nlp(article)

# useful_entities = [u'NORP', u'PERSON', u'FACILITY', u'ORG', u'GPE', u'LOC', u'EVENT', u'DATE', u'TIME',u'NUM']
# for element in document.ents:
#     if (element.label_ in useful_entities):
#         result.append(element)
# print('Original Sentence: %s' % (article))
# for element in document.ents:
#     print('Type: %s, Value: %s' % (element.label_, element))
# print(result)

def get_NER(tweet):
    result = []
    spacy_nlp = spacy.load('en')
    document = spacy_nlp(tweet)
    useful_entities = [u'NORP', u'PERSON', u'FACILITY', u'ORG', u'GPE', u'LOC', u'EVENT', u'DATE', u'TIME',u'NUM']
    for element in document.ents:
        if (element.label_ in useful_entities):
            result.append(str(element))
#     print("hello")
    return result

In [ ]:
dfchange = df.loc[df["DateHour"]==list_date[0]]
# dfchange = dfchange[0:4]
# dfchange

In [ ]:
# function to get unique values 
def getNumClusters(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return len(unique_list)

# take second element for sort
def takeSecond(elem):
    return elem[1]

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from multiprocessing import Pool

num_partitions = 4 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [ ]:
def my_fun(data_pd):
    data_pd['ner'] = data_pd["tweets"].map(lambda x: get_NER(x))
    return data_pd

def parallelize_dataframe(df, my_func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(my_func, df_split))
    pool.close()
    pool.join()
    return df

final_df = parallelize_dataframe(dfchange,my_fun)

In [ ]:
#print(final_df["ner"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def getTfIdf(clean_train,windows_corpus):
    minimum = max(int(windows_corpus * 0.0025), 10)
    print(minimum)
    tfidf_vectorizer = TfidfVectorizer(#min_df=0.5,
                                       min_df=minimum,
                                       #max_df=0.8,
                                       max_features=2000000,
                                       stop_words='english',
                                       use_idf=True,
                                       ngram_range=(1,3)
    )
    # Tf-idf-weighted term-document sparse matrix
    try :
        tfidf_train_data_features = tfidf_vectorizer.fit_transform(clean_train)
#         print(tfidf_vectorizer.get_feature_names())
    except:
        return []
    

    #sim = (tfidf_train_data_features * tfidf_train_data_features.T).A
    return tfidf_train_data_features

In [ ]:
def findTweetsDBSCAN(tfidf,e):
    clustering = DBSCAN(eps=e, min_samples=10,n_jobs=-1).fit(tfidf)
    #clustering = hdbscan.HDBSCAN(min_cluster_size=10).fit(tfidf)

    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
    cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
        
    #Noise Data
    noise_data = addedCluster[addedCluster['Cluster'] == -1]
#     print("Size:",noise_data.shape[0])
#     print(noise_data["tweets"],noise_data["Event"])
    
    return {"clusters":clusters,"noise_data":noise_data}

In [ ]:
# Function to convert   
def listToString(s):
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

final_df["total_sentence"] = np.array([listToString(temp) for temp in final_df['ner']])

In [ ]:
tweets = final_df
tweetsContent = tweets.copy()["total_sentence"]
tfidf = getTfIdf(tweetsContent,dfchange.shape[0])
print("Event",count,":")
count+=1
print("Tweets: ",dfchange.shape[0])
x_object = findTweetsDBSCAN(tfidf,0.8)
cluster = x_object["clusters"]
noise_data = x_object["noise_data"]

for i in range(0,len(cluster)):
    temp = cluster[i].set_index(["Event","id"]).count(level="Event")
    temp2 = temp.loc[temp["Cluster"]==temp["Cluster"].max()]
    predicted_event = temp2.index[0]
    
    for index,row in cluster[i].iterrows():
        #row["Predicted_Event"] = predicted_event
        list_y.append(row["Event"])
        list_y_preticted.append(predicted_event)
#     print("Predicted Event:", -1)

for i in range(0,len(cluster)):
    print(print("Cluster", i, ":", cluster[i].shape[0]))
    print(cluster[i]["tweets"],cluster[i]["Event"])
    sum+= cluster[i].shape[0]
    #print("\n")

# print("Noise data:",noise_data["tweets"],noise_data["Event"])
for index,row in noise_data.iterrows():
    list_y.append(row["Event"])
    list_y_preticted.append(-1)

print("Number of clusters", len(cluster))
print("Total tweets that clustered",sum)
print("\n")

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import f1_score
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import normalized_mutual_info_score

print(accuracy_score(list_y,list_y_preticted))
print(f1_score(list_y,list_y_preticted,average='micro'))
print(adjusted_rand_score(list_y,list_y_preticted))
print(adjusted_mutual_info_score(list_y,list_y_preticted))
print(normalized_mutual_info_score(list_y, list_y_preticted))